<a href="https://colab.research.google.com/github/zzaeyun/ESAA23_1/blob/main/%EC%99%84%EB%B2%BD%EA%B0%80%EC%9D%B4%EB%93%9C_497to512.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



> 파이썬 완벽가이드 p.497~512

## **05 감성분석**

###**감성 분석 소개**

감성분석(Sentiment Analysis)
- 문서의 주관적인 감성/의견/감정/기분 등을 파악하기 위한 방법으로 소셜 미디어, 여론조사, 온라인 리뷰, 피드백 등 다양한 분야에서 활용
- 문서 내 텍스트가 나타내는 여러 가지 주관적인 단어와 문맥을 기반으로 감성(Sentiment) 수치를 계산하는 방법
  - 감성 지수는 긍정 감성 지수와 부정 감성 지수로 구성되며 이들 지수를 합산해 긍정 감성 또는 부정 감성을 결정

<br>
**지도학습**
- 학습 데이터와 타깃 레이블 값을 기반으로 감성 분석 학습을 수행한 뒤 이를 기반으로 다른 데이터의 감성 분석을 예측하는 방법으로 일반적인 텍스트 기반의 분류와 거의 동일
**비지도학습**
- 'Lexicon'이라는 일종의 감성 어휘 사전을 이용. 
- Lexicon은 감성 분석을 위한 용어와 문맥에 대한 다양한 정보를 가지고 있으며, 이를 이용해 문서의 긍정적, 부정적 감성 여부를 판단






### **지도학습 기반 감성 분석 실습 - IMDB 영화평**
- IMDB의 영화 사이트의 영화평을 이용
- 영화평의 텍스트를 분석해 감성 분석 결과가 긍정 또는 부정인지를 예측하는 모델 생성


In [10]:
import pandas as pd

review_df=pd.read_csv('/labeledTrainData.tsv', header=0, sep='\t', quoting=3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


- id: 각 데이터의 id
- sentiment: 영화평의 Sentiment 결과 값(Target Label). 1은 긍정적 평가. 0은 부정적 평가를 의미
- review: 영화평의 텍스트


In [11]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

HTML 형식에서 추출하여 &lt;br /&gt; 태그가 여전히 존재
- &lt;br /&gt; 문자열은 피처로 만들 필요가 없으니 삭제
- 판다스의 DataFrame/Series는 문자열 연산을 지원하기 위해 str 속성을 이용하므로 replace()를 str에 적용해 &lt;br /&gt; 태그를 변경

<br>

- 영어가 아닌 숫자/특수문자 역시 Sentiment를 위한 피처로는 의미가 없으므로 공란으로 변경

In [12]:
import re

review_df['review']=review_df['review'].str.replace('<br />', ' ')

#파이썬의 정규 표현식 모듈인 re를 이용해 영어 문자열이 아닌 문자는 모두 공백으로 변환
review_df['review']=review_df['review'].apply( lambda x : re.sub('[^a-zA-z]',' ',x))

In [13]:
from sklearn.model_selection import train_test_split

class_df=review_df['sentiment']
feature_df=review_df.drop(['id', 'sentiment'], axis=1, inplace=False)

X_train, X_test, y_train, y_test=train_test_split(feature_df, class_df, test_size=0.3, random_state=156)
X_train.shape, X_test.shape

((17500, 1), (7500, 1))

- 감상평 텍스트를 피처 벡터화한 후에 ML 분류 알고리즘을 적용해 예측 성능을 측정
- Pipeline 객체를 이용해 두 가지를 한꺼번에 수행
  - Count 벡터화를 적용해 예측 성능을 측정
  - TF-IDF 벡터화를 적용
- Classifier는 LogisticRegression 이용

In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipeline=Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2) )),
     ('lr_clf', LogisticRegression(C=10))])

In [15]:
pipeline.fit(X_train['review'], y_train)
pred=pipeline.predict(X_test['review'])
pred_probs=pipeline.predict_proba(X_test['review'])[:,1]

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test, pred),
                                                 roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8860, ROC-AUC는 0.9502


- TF-IDF 벡터화 적용

In [17]:
pipeline=Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=10))
])

pipeline.fit(X_train['review'], y_train)
pred=pipeline.predict(X_test['review'])
pred_probs=pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test, pred),
                                                 roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8935, ROC-AUC는 0.9598


TF-IDF 기반 피처 벡터화의 예측 성능이 조금 더 나아짐

---



### **비지도학습 기반 감성 분석 소개**
- Lexicon을 기반: 한글 지원X
- Lexicon은 일반적으로 어휘집을 의미. 여기서는 주로 감성만을 분석하기 위해 지원하는 감성 어휘 사전: 감성 사전
  - 감성 사전은 긍정(Positive) 감성 또는 부정(Negative)감성의 정도를 의미하는 수치를 가지고 있음: 이를 감성 지수(Polarity score)라고 함
  - 감성 지수는 단어의 위치나 주변 단어, 문맥, POS(Part of Speech) 등을 참고하여 결정
- NLTK 패키지
  - 감성 사전을 구현한 패키지
  - Lexicon 모듈을 포함하여 많은 서브 모듈을 가지고 있음

<br>

**WordNet**
- 방대한 영어 어휘 사전
  - 단순한 어휘 사전이 아닌 시맨틱 분석을 제공하는 어휘 사전
  - **시맨틱(semantic)**: 문맥상 의미
- 다양한 상황에서 같은 어휘라도 다르게 사용되는 어휘의 시맨틱 정보를 제공하며, 이를 위해 각각의 품사로 구성된 개별 단어를 Synset(Sets of cogmitive synonyms)이라는 개념을 이용하여 표현
  - Synset은 단순히 하나의 단어가 아니라 그 단어가 가지는 문맥, 시맨틱 정보를 제공하는 WordNet의 핵심 개념

<br>

**NLTK를 포함한 대표적인 감성 사전**
- SentiWordNet: NLTK 패키지의 WordNet과 유사하게 감성 단어 전용의 WordNet을 구현한 것
  - 3가지 감성 점수 할당: 긍정 감성 지수, 부정 감성 지수, 객관성 지수
- VADER: 주로 소셜 미디어의 텍스트에 대한 감성 분석을 제공하기 위한 패키지. 뛰어난 감성 분석 결과를 제공하며, 비교적 빠른 수행 시간을 보장해 대용량 텍스트 데이터에 잘 사용되는 패키지
- Pattern: 예측 성능 측면에서 가장 주목받는 패키지. 아쉽께도 파이썬 3.X 버전에서 호환 불가능


###**SentiWordNet을 이용한 감성 분석**

**WordNet Synset과 SentiWordNet SentiSynset 클래스의 이해**

In [18]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

NLTK의 모든 데이터 세트를 내려받은 뒤에 WordNet 모듈을 임포트해서 'present'단어에 대한 Synset을 추출
- WordNet의 synsets()는 파라미터로 지정된 단어에 대해 WordNet에 등재된 모든 Synset 객체를 반환

In [19]:
from nltk.corpus import wordnet as wn

term='present'

synsets=wn.synsets(term)
print('synsets() 반환 type: ', type(synsets))
print('synsets() 반환 값 개수: ', len(synsets))
print('synsets() 반환 값: ', synsets)

synsets() 반환 type:  <class 'list'>
synsets() 반환 값 개수:  18
synsets() 반환 값:  [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


- synset( ) 호출 시 반환되는 것은 여러 개의 Synset 객체를 가지는 리스트
- 총 18개의 서로 다른 semantic을 가지는 synset 객체가 반환

<br>

- synset은 POS(품사), 정의(Definition), 부명제(Lemma) 등으로 시맨틱적인 요소를 표현 가능


In [20]:
for synset in synsets:
  print('##### Synset name: ', synset.name(), '#####')
  print('POS: ', synset.lexname())
  print('Definition: ', synset.definition())
  print('Lemmas :', synset.lemma_names())

##### Synset name:  present.n.01 #####
POS:  noun.time
Definition:  the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas : ['present', 'nowadays']
##### Synset name:  present.n.02 #####
POS:  noun.possession
Definition:  something presented as a gift
Lemmas : ['present']
##### Synset name:  present.n.03 #####
POS:  noun.communication
Definition:  a verb tense that expresses actions or states at the time of speaking
Lemmas : ['present', 'present_tense']
##### Synset name:  show.v.01 #####
POS:  verb.perception
Definition:  give an exhibition of to an interested audience
Lemmas : ['show', 'demo', 'exhibit', 'present', 'demonstrate']
##### Synset name:  present.v.02 #####
POS:  verb.communication
Definition:  bring forward and present to the mind
Lemmas : ['present', 'represent', 'lay_out']
##### Synset name:  stage.v.01 #####
POS:  verb.creation
Definition:  perform (a play), especially on a stage
Lemmas : ['stage', 'present', 're

- Synset('present.n.01')과 Synset('present.n.02')는 명사지만 서로 다른 의미를 가지고 있음

<br>

- WordNet은 어떤 어휘와 다른 어휘 간의 관계를 유사도로 나타낼 수 있음
- synset 객체는 단어 간의 유사도를 나타내기 위해서 path_similarity()메서드를 제공


In [21]:
tree=wn.synset('tree.n.01')
lion=wn.synset('lion.n.01')
tiger=wn.synset('tiger.n.02')
cat=wn.synset('cat.n.01')
dog=wn.synset('dog.n.01')

entities=[tree, lion, tiger, cat, dog]
similarities=[]

entity_names=[entity.name().split('.')[0] for entity in entities]

for entity in entities:
  similarity = [round(entity.path_similarity(compared_entity),2)
                    for compared_entity in entities]
  similarities.append(similarity)

In [22]:
similarity_df=pd.DataFrame(similarities, columns=entity_names, index=entity_names)
similarity_df

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


SentiWordNet은 WordNet의 Synset과 유사한 Senti_Synset 클래스를 가지고 있음
- SentiWordNet 모듈의 senti_synsets()는 WordNet 모듈이라서 synsets()과 비슷하게 Senti_Synest 클래스를 리스트 형태로 반환

In [23]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets=list(swn.senti_synsets('slow'))
print('senti_synsets() 반환 type: ', type(senti_synsets))
print('senti_synsets() 반환 값 개수: ', len(senti_synsets))
print('senti_synsets() 반환 값: ', senti_synsets)

senti_synsets() 반환 type:  <class 'list'>
senti_synsets() 반환 값 개수:  11
senti_synsets() 반환 값:  [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


SentiWordNet 객체의 객관성 지수
- 어떤 단어가 전혀 감정적이지 않으면 객관성 지수는 1이 되고, 감성 지수는 모두 0이 됨

In [25]:
import nltk
from nltk.corpus import sentiwordnet as swn

father=swn.senti_synset('father.n.01')
print('father 긍정감성 지수: ', father.pos_score())
print('father 부정감성 지수: ', father.neg_score())
print('father 객관성 지수: ', father.obj_score())

print('\n')

fabulous=swn.senti_synset('fabulous.a.01')
print('fabulous 긍정감성 지수: ', fabulous.pos_score())
print('fabulous 부정감성 지수: ', fabulous.neg_score())

father 긍정감성 지수:  0.0
father 부정감성 지수:  0.0
father 객관성 지수:  1.0


fabulous 긍정감성 지수:  0.875
fabulous 부정감성 지수:  0.125


**SentiWordNet을 이용한 영화 감상평 감성 분석**
1. 문서(Document)를 문장(Sentence) 단위로 분해
2. 다시 문장을 단어(Word) 단위로 토큰화하고 품사 태깅
3. 품사 태깅된 단어 기반으로 synset 객체와 senti_synset 객체를 생성
4. Senti_synset에서 긍정 감성/부정 감성 지수를 구하고 이를 모두 합산해 특정 임계치 값 이상일 때 긍정 감성으로, 그렇지 않을 때는 부정감성으로 결정



SentiWordNet을 이용하기 위해기 WordNet을 이용해 문서를 다시 단어로 토큰화한 뒤 어근 추출(Lemmatization)과 품사 태깅(POS Tagging)을 적용해야 함

In [26]:
from nltk.corpus import wordnet as wn

def penn_to_wn(tag):
  if tag.startswith('J'):
    return wn.ADJ
  elif tag.startswith('N'):
    return wn.NOUN
  elif tag.startswith('R'):
    return wn.ADV
  elif tag.startswith('V'):
    return wn.VERB

In [28]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
  sentiment=0.0
  tokens_count=0

  lemmatizer = WordNetLemmatizer()
  raw_sentences=sent_tokenize(text)

  for raw_sentence in raw_sentences:
    tagged_sentence=pos_tag(word_tokenize(raw_sentence))
    for word, tag in tagged_sentence:
      wn_tag=penn_to_wn(tag)
      if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        continue
      lemma=lemmatizer.lemmatize(word, pos=wn_tag)
      if not lemma:
        continue
      synsets=wn.synsets(lemma, pos=wn_tag)
      if not synsets:
        continue
      
      synset=synsets[0]
      swn_synset=swn.senti_synset(synset.name())
      sentiment +=(swn_synset.pos_score() - swn_synset.neg_score())
      tokens_count += 1

  if not tokens_count:
    return 0

  if sentiment >=0:
    return 1

  return 0

swn_polarity(text) 함수를 IMDB 감상평의 개별 문서에 적용해 긍정 및 부정 감성을 예측

In [30]:
train_df=review_df

In [31]:
 train_df['preds']=train_df['review'].apply(lambda x: swn_polarity(x))
 y_target=train_df['sentiment'].values
 preds=train_df['preds'].values

SentiWordNet의 감성 분석 예측 성능 살펴보기

In [32]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

print(confusion_matrix(y_target, preds))
print('정확도: ', np.round(accuracy_score(y_target, preds),4))
print('정밀도: ', np.round(precision_score(y_target, preds),4))
print('재현율: ', np.round(recall_score(y_target, preds),4))

[[7685 4815]
 [3621 8879]]
정확도:  0.6626
정밀도:  0.6484
재현율:  0.7103


###**VADER를 이용한 감성 분석**

- VADER는 소셜 미디어의 감성 분석 용도로 만들어진 룰 기반의 Lexicon
- SentimentIntensityAnalyzer 클래스를 이용해 쉽게 감성 분석을 제공


In [33]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer=SentimentIntensityAnalyzer()
senti_scores=senti_analyzer.polarity_scores(train_df['review'][0])
print(senti_scores)

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}


1. SentimentIntensityAnalyzer 객체를 생성한 뒤에 문서별로 polarity_scores( )메서드를 호출해 감성 점수를 구함
2. 해당 문서의 감성 점수가 특정 임계값 이상이면 긍정, 그렇지 않으면 부정으로 판단
  - SentimentIntensity Analyzer 객체의 polarity_scores()메서드는 딕셔너리 형태의 감성 점수를 반환
    - neg: 부정 감성 지수
    - neu: 중립적인 감성 지수
    - pos: 긍정 감성 지수
    - compound: neg, neu, pos score를 적절히 조합해 -1에서 1 사이의 감성 지수를 표현한 값

    

In [34]:
def vader_polarity(review, threshold=0.1):
  analyzer=SentimentIntensityAnalyzer()
  scores=analyzer.polarity_scores(review)

  agg_score=scores['compound']
  final_sentiment=1 if agg_score>=threshold else 0
  return final_sentiment

review_df['vader_preds']=review_df['review'].apply(lambda x:vader_polarity(x, 0.1))
y_target=review_df['sentiment'].values
vader_preds=review_df['vader_preds'].values

print(confusion_matrix(y_target, vader_preds))
print('정확도: ', np.round(accuracy_score(y_target, vader_preds), 4))
print('정밀도: ', np.round(precision_score(y_target, vader_preds), 4))
print('재현율: ', np.round(recall_score(y_target, vader_preds), 4))

[[ 6762  5738]
 [ 1842 10658]]
정확도 : 0.6968
정밀도 : 0.65
재현율 : 0.8526
